In [21]:
import json
import openai


client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-5-nano-2025-08-07",
    messages=[
        {
            "role": "user",
            "content": "what is the capital of Greece",
        }
    ]
)


print(json.dumps(response.model_dump(), indent=4, ensure_ascii=False))

messages = response.choices[0].message

print (messages)

{
    "id": "chatcmpl-CL2KwywqU3mqWDv1rHQYmxLvTQmLS",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "Athens. It’s the capital and largest city of Greece. Would you like a quick overview of notable sights there?",
                "refusal": null,
                "role": "assistant",
                "annotations": [],
                "audio": null,
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1759129906,
    "model": "gpt-5-nano-2025-08-07",
    "object": "chat.completion",
    "service_tier": "default",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 161,
        "prompt_tokens": 12,
        "total_tokens": 173,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 128,
  